In [4]:
import pm4py
import os
import pandas as pd
import numpy as np
import copy
from collections import Counter
import random
import re
import warnings
warnings.filterwarnings("ignore")

In [7]:
def csv_to_xes(file, add_startend, id_column, act_column, start_time_column, end_time_column, resource_column):
    import_file = f'{file}.csv' #
    df = pd.read_csv(import_file)
#     print(df)
    id_column = id_column
    act_column = act_column
    start_time_column = start_time_column
    end_time_column = end_time_column
    resource_column = resource_column
    state_column = 'lifecycle:transition'
    time_column = 'time:timestamp'
    df[resource_column].fillna('no_resource', inplace=True)
    df.to_csv(rf'C:\Users\19wuj\PycharmProjects\CoDi\tabular_datasets\diffu\PurchasingExample_0.2\{file}.csv', index=False)
    if add_startend:
        # pre-processing
        mydata = pd.DataFrame()
        for key, group in df.groupby(id_column):
            group = add_start_end(df, group, act_column, resource_column, start_time_column, end_time_column)   # add start event in the beginning and end in the last 
            mydata = pd.concat([mydata, group], ignore_index=True)
        df = mydata

    
    df_start = df[[id_column, act_column, start_time_column, resource_column]].rename(columns={start_time_column:time_column})
    df_start[state_column] = 'start'
    df_start

    df_end = df[[id_column, act_column, end_time_column, resource_column]].rename(columns={end_time_column:time_column})
    df_end[state_column] = 'complete'

    df_start_end = pd.concat([df_start, df_end])
    df_start_end['index'] = df_start_end.index
    df_start_end[time_column] = pd.to_datetime(df_start_end[time_column])
    new = pd.DataFrame()
    for key, group in df_start_end.groupby(id_column):
        temp = group.sort_values(by=[time_column, 'index'], ascending=[True, True]).reset_index(drop=True)
        new = pd.concat([new,temp],ignore_index = True)
    del new['index']
    df = new
    
    
#     print(new.columns)
    df['concept:name'] = df[act_column].astype('str')
    df[id_column] = df[id_column].astype('str')
    print(df)
    df = df[[id_column, 'concept:name', state_column, resource_column, time_column, act_column]]
#     df = df[[id_column, act_column,  state_column, resource_column, time_column]]
    df = df.rename(columns = { resource_column: 'user', act_column: 'task'})
    
    pm4py.write.write_xes(df, rf'C:\Users\19wuj\PycharmProjects\CoDi\tabular_datasets\diffu\PurchasingExample_0.2\{file}.xes', case_id_key=id_column)

In [8]:
csv_to_xes('train_PurchasingExample', add_startend = False, id_column='caseid', act_column='task', start_time_column='start_timestamp', end_time_column='end_timestamp', resource_column='user')

      caseid                          task      time:timestamp  \
0          2                         Start 2011-01-01 00:16:00   
1          2                         Start 2011-01-01 00:16:00   
2          2   Create Purchase Requisition 2011-01-01 00:16:00   
3          2   Create Purchase Requisition 2011-01-01 00:29:00   
4          2  Create Request for Quotation 2011-01-01 08:16:00   
...      ...                           ...                 ...   
16373   1949   Create Purchase Requisition 2011-10-13 07:21:00   
16374   1949  Analyze Purchase Requisition 2011-10-13 17:03:00   
16375   1949  Analyze Purchase Requisition 2011-10-13 17:09:00   
16376   1949                           End 2011-10-13 17:09:00   
16377   1949                           End 2011-10-13 17:09:00   

                      user lifecycle:transition                  concept:name  
0                    Start                start                         Start  
1                    Start             complete

exporting log, completed traces ::   0%|          | 0/486 [00:00<?, ?it/s]

In [36]:
# info of train_data
if_csv = False #
import_file = rf'C:\Users\19wuj\PycharmProjects\CoDi\tabular_datasets\diffu\PurchasingExample_0.2\train_PurchasingExample.xes' #
id_column = 'caseid'
act_column = 'concept:name'
time_column = 'time:timestamp'
resource_column = 'user'
state_column = 'lifecycle:transition'

dirStr, ext = os.path.splitext(import_file)
file_name = dirStr.split("\\")[-1]


data = pm4py.read.read_xes(import_file)

df = copy.deepcopy(pd.DataFrame(data,columns=[id_column, act_column, time_column, resource_column, state_column]))

parsing log, completed traces ::   0%|          | 0/486 [00:00<?, ?it/s]

In [12]:
df['waiting_time'] = None
df['process_time'] = None
df['last_complete_event'] = None
df['preceding_evts'] = None
df['paired_event'] = None
df['next'] = None
df['index'] = df.index
for key, group in df.groupby(id_column):
    flag = 0
    i = list(group.index)[0]
    preceding_evt = []
    not_complete_evt_idx = []
    j = 0
    df['waiting_time'].loc[i] = 0
    not_complete_evt_idx.append(i)
    i+=1
    while j<len(group)-1:
        j+=1
        cur_act = df.loc[i]
        if cur_act[state_column] == 'complete':
            flag = 0
            preceding_evt.append(cur_act.name)
            last_complete_evt_idx = i
            
            for each_idx in not_complete_evt_idx:
                to_pair = df.loc[each_idx]
                if (cur_act[act_column] == to_pair[act_column]) \
                and (cur_act[resource_column] == to_pair[resource_column]):
                    df['paired_event'].loc[i] = each_idx
                    not_complete_evt_idx.remove(each_idx)
                    df['process_time'].loc[i] = (df[time_column].loc[i] - df[time_column].loc[each_idx]).total_seconds()
                    df['last_complete_event'].loc[i] = df['last_complete_event'].loc[each_idx]
                    break
            
        else:
            if flag == 1:
                df['preceding_evts'].loc[i] = df['preceding_evts'].loc[i-1]
            else:
                df['preceding_evts'].loc[i] = preceding_evt#tuple(sorted, reverse=False))
            for each in df['preceding_evts'].loc[i]:
                if df['next'].loc[each] == None:
                    df['next'].loc[each] = [i]
                else:
                    temp = df['next'].loc[each]
                    temp.append(i)
                    df['next'].loc[each] = temp
            flag = 1
            preceding_evt = []
            not_complete_evt_idx.append(i)
            
            df['last_complete_event'].loc[i] = last_complete_evt_idx
            df['waiting_time'].loc[i] = (df[time_column].loc[i] - df[time_column].loc[last_complete_evt_idx]).total_seconds()
        i+=1

In [13]:
def fillna_parallel_data(bb):
    i = bb.index[0]
    j = 0 
    while j < len(bb):
        if bb[state_column].loc[i] == 'complete':
            paired = int(bb['paired_event'].loc[i])
            bb['process_time'].loc[paired] = bb['process_time'].loc[i] 
            bb['paired_event'].loc[paired] = i
            bb['next'].loc[paired] = bb['next'].loc[i]
            bb['waiting_time'].loc[i] = bb['waiting_time'].loc[paired] 
            bb['preceding_evts'].loc[i] = bb['preceding_evts'].loc[paired]
            bb['next'].loc[paired] = bb['next'].loc[i] 
        i+=1
        j+=1
fillna_parallel_data(df)


In [14]:
df = df[df[state_column]=='start']
df = df.rename(columns = {'concept:name':'activity'})
file = df[['activity', 'user', 'waiting_time', 'process_time']]
file

,activity,user,waiting_time,process_time
0,Start,Start,0,0
2,Create Purchase Requisition,Immanuel Karagianni,0,780
4,Create Request for Quotation,Alberto Duport,28020,600
6,Analyze Request for Quotation,Karel de Groot,4080,240
8,Amend Request for Quotation,Christian Francois,2280,300
...,...,...,...,...
16368,End,End,0,0
16370,Start,Start,0,0
16372,Create Purchase Requisition,Tesca Lobes,0,3240
16374,Analyze Purchase Requisition,Francis Odell,34920,360


In [15]:
import math
file['waiting_time'] = file['waiting_time'].map(lambda x: math.log(x+1))
file['process_time'] = file['process_time'].map(lambda x: math.log(x+1))

In [16]:
file.reset_index(drop=True, inplace = True)
file

,activity,user,waiting_time,process_time
0,Start,Start,0.000000,0.000000
1,Create Purchase Requisition,Immanuel Karagianni,0.000000,6.660575
2,Create Request for Quotation,Alberto Duport,10.240710,6.398595
3,Analyze Request for Quotation,Karel de Groot,8.314097,5.484797
4,Amend Request for Quotation,Christian Francois,7.732369,5.707110
...,...,...,...,...
8184,End,End,0.000000,0.000000
8185,Start,Start,0.000000,0.000000
8186,Create Purchase Requisition,Tesca Lobes,0.000000,8.083637
8187,Analyze Purchase Requisition,Francis Odell,10.460844,5.888878


In [17]:
from collections import Counter
from sklearn import preprocessing
new_data = pd.DataFrame()
def get_info(df, new_df, column_name, column_type):
    if column_type == 'categorical':
        df[column_name].fillna(f'NO_{column_name}',inplace=True)
        dict_ = {}
        a = Counter(df[column_name])     
        dict_['name'] = column_name
        dict_['size'] = len(a.keys())
        dict_['type'] = column_type
        enc=preprocessing.LabelEncoder()
        enc=enc.fit(list(a.keys()))#训练LabelEncoder,将电脑，手表，手机编码为0,1,2
        data=enc.transform(df[column_name])#使用训练好的LabelEncoder对原数据进行编码，也叫归一化
        dict_['i2s'] = list(enc.classes_)
        new_df[column_name] = data
        return dict_
    else:
        dict_ = {}
        dict_['max'] = max(df[column_name])
        dict_['min'] = min(df[column_name])
        dict_['name'] = column_name
        dict_['type'] = column_type
        new_df[column_name] = df[column_name]
        return dict_


In [18]:
new_data = pd.DataFrame()
a = [ 'activity']
b = ['waiting_time', 'process_time']

json_input = {}
json_input['columns'] = [get_info(file, new_data, each, 'categorical') for each in a]
json_input['columns'].extend([get_info(file, new_data, each, 'continuous') for each in b])
json_input['problem_type'] = 'no'

In [19]:
new_data

,activity,waiting_time,process_time
0,22,0.000000,0.000000
1,10,0.000000,6.660575
2,12,10.240710,6.398595
3,4,8.314097,5.484797
4,1,7.732369,5.707110
...,...,...,...
8184,14,0.000000,0.000000
8185,22,0.000000,0.000000
8186,10,0.000000,8.083637
8187,2,10.460844,5.888878


In [20]:
for key, group in new_data.groupby('activity'):
    if key == 6.0:
        print(group)

      activity  waiting_time  process_time
21           6      0.000000           0.0
44           6      0.000000           0.0
68           6      0.000000           0.0
105          6      6.293419           0.0
140          6      0.000000           0.0
...        ...           ...           ...
7932         6      0.000000           0.0
7959         6      0.000000           0.0
8015         6      0.000000           0.0
8041         6      0.000000           0.0
8066         6      0.000000           0.0

[325 rows x 3 columns]


In [21]:
train_number = round(len(new_data)*0.8)
train_number

6551

In [24]:
import json
with open(rf'C:\Users\19wuj\PycharmProjects\CoDi\tabular_datasets\diffu\PurchasingExample_0.2\PurchasingExample_act_time.json',"w") as f:
    json.dump(json_input,f)

# with open('test.json') as json_file:
#     a = json.load(json_file)

train = np.array(new_data.iloc[:train_number]).astype('float32')
train.shape

test = np.array(new_data.iloc[train_number:]).astype('float32')
test.shape


array_dict = {'train':train, 'test':test}
np.savez(rf'C:\Users\19wuj\PycharmProjects\CoDi\tabular_datasets\diffu\PurchasingExample_0.2\PurchasingExample_act_time.npz', **array_dict)




In [25]:
gen = pd.read_csv('gen_seq_train_PurchasingExample.csv')
gen_act = gen['act']
gen_act = gen_act.map(lambda x:json_input['columns'][0]['i2s'].index(x))
gen_act = np.array(gen_act).astype('float32')
gen_act

array_dict = {'gen':gen_act}
np.savez('PurchasingExample_act_time_gen.npz', **array_dict)

In [107]:
gen_diffu_time = pd.read_csv(rf'C:\Users\19wuj\PycharmProjects\CoDi\exp_PurchasingExample_act_time/sample_4.csv')
gen_diffu_time = gen_diffu_time.rename(columns = {'0':'activity', '1':'waiting_time', '2':'process_time'})
gen_diffu_time['wait'] = gen_diffu_time['waiting_time'].map(lambda x: round(math.exp(x)-1))
gen_diffu_time['pro'] = gen_diffu_time['process_time'].map(lambda x: round(math.exp(x)-1))
gen_diffu_time

,activity,waiting_time,process_time,wait,pro
0,22.0,0.086415,0.076116,0,0
1,10.0,0.146860,6.808235,0,904
2,12.0,6.704126,5.424286,815,226
3,4.0,13.829016,5.998834,1013596,402
4,19.0,10.452233,7.450936,34621,1720
...,...,...,...,...,...
1965,12.0,6.125717,4.454077,456,85
1966,4.0,12.451329,7.400385,255589,1636
1967,1.0,8.195540,6.104255,3624,447
1968,4.0,12.295001,7.082032,218599,1189


In [101]:
[int(each) for each in '(32,34)'.strip('(').strip(')').split(',')]

[32, 34]

In [108]:
gen = pd.read_csv(r'C:\Users\19wuj\PycharmProjects\CoDi\tabular_datasets\diffu\PurchasingExample_0.2/gen_seq_train_PurchasingExample.csv')
gen['pro'] = gen_diffu_time['pro']
gen['wait'] = gen_diffu_time['wait']
gen = gen.set_index('Unnamed: 0')
gen['last'] = gen['last'].map(lambda x: [int(each) for each in x.strip('(').strip(')').split(',')] if x!='None' else x)
gen['next'] = gen['next'].map(lambda x: [int(each) for each in x.strip('(').strip(')').split(',')] if x!='None' else x)
# gen['last'] = gen['last'].map(lambda x: [x])
# gen['next'] = gen['next'].map(lambda x: [x])
gen

,caseid,act,index,last,next,if_para,start_timestamp,end_timestamp,resource,pro,wait
Unnamed: 0,,,,,,,,,,,
1,1,Start,1,None,[2],0,NaN,NaN,Start,0,0
2,1,Create Purchase Requisition,2,[1],[3],0,NaN,NaN,Create Purchase Requisition,904,0
3,1,Create Request for Quotation,3,[2],[4],0,NaN,NaN,Create Request for Quotation,226,815
4,1,Analyze Request for Quotation,4,[3],[5],0,NaN,NaN,Analyze Request for Quotation,402,1013596
5,1,Send Request for Quotation to Supplier,5,[4],[6],0,NaN,NaN,Send Request for Quotation to Supplier,1720,34621
...,...,...,...,...,...,...,...,...,...,...,...
2208,122,Create Request for Quotation,2208,[2207],[2209],0,NaN,NaN,Create Request for Quotation,85,456
2209,122,Analyze Request for Quotation,2209,[2208],[2210],0,NaN,NaN,Analyze Request for Quotation,1636,255589
2210,122,Amend Request for Quotation,2210,[2209],[2211],0,NaN,NaN,Amend Request for Quotation,447,3624


In [109]:
# get the start time of each generated trace
ia_df = pd.read_csv(r'C:/Users/19wuj/PycharmProjects/CoDi/tabular_datasets/diffu/PurchasingExample_0.2/gen_PurchasingExample_1.csv')
ia_df
ia_list = []
for key, group in ia_df.groupby('caseid'):
    group['start_timestamp'] = group['start_timestamp'].map(lambda x: pd.to_datetime(x, format = '%Y-%m-%d  %H:%M:%S'))
    ia_list.append(min(group['start_timestamp']))
len(ia_list)

122

In [110]:
g = pd.DataFrame()
for key, group in gen.groupby('caseid'):
    group = group.set_index('index')
    for i in range(len(group)):
        cur = group.iloc[i]
        if i == 0:
            group['start_timestamp'].iloc[i] = ia_list[int(key)-1]
            group['end_timestamp'].iloc[i] = group['start_timestamp'].iloc[i] + pd.Timedelta(seconds=0)
        else:
            if len(cur['last']) > 1:
                last_time = max(pd.to_datetime(group['end_timestamp'].loc[list(cur['last'])]))
                group['start_timestamp'].iloc[i] = last_time + pd.Timedelta(seconds=group['wait'].iloc[i])
            else:
                last_time = pd.to_datetime(group['end_timestamp'].loc[cur['last'][0]])
                group['start_timestamp'].iloc[i] = last_time + pd.Timedelta(seconds=group['wait'].iloc[i])
            group['end_timestamp'].iloc[i] = group['start_timestamp'].iloc[i] + pd.Timedelta(seconds=group['pro'].iloc[i])
    g = pd.concat([g, group])
g

,caseid,act,last,next,if_para,start_timestamp,end_timestamp,resource,pro,wait
index,,,,,,,,,,
1,1,Start,None,[2],0,2011-01-01 02:28:58,2011-01-01 02:28:58,Start,0,0
2,1,Create Purchase Requisition,[1],[3],0,2011-01-01 02:28:58,2011-01-01 02:44:02,Create Purchase Requisition,904,0
3,1,Create Request for Quotation,[2],[4],0,2011-01-01 02:57:37,2011-01-01 03:01:23,Create Request for Quotation,226,815
4,1,Analyze Request for Quotation,[3],[5],0,2011-01-12 20:34:39,2011-01-12 20:41:21,Analyze Request for Quotation,402,1013596
5,1,Send Request for Quotation to Supplier,[4],[6],0,2011-01-13 06:18:22,2011-01-13 06:47:02,Send Request for Quotation to Supplier,1720,34621
...,...,...,...,...,...,...,...,...,...,...
2208,122,Create Request for Quotation,[2207],[2209],0,2011-01-06 13:06:42,2011-01-06 13:08:07,Create Request for Quotation,85,456
2209,122,Analyze Request for Quotation,[2208],[2210],0,2011-01-09 12:07:56,2011-01-09 12:35:12,Analyze Request for Quotation,1636,255589
2210,122,Amend Request for Quotation,[2209],[2211],0,2011-01-09 13:35:36,2011-01-09 13:43:03,Amend Request for Quotation,447,3624


In [111]:
g = g.rename(columns = {'act':'task'})
g['caseid'] = g['caseid'].map(lambda x: 'Case'+str(x))
g = g[['caseid', 'task','start_timestamp', 'end_timestamp', 'resource']]
g

,caseid,task,start_timestamp,end_timestamp,resource
index,,,,,
1,Case1,Start,2011-01-01 02:28:58,2011-01-01 02:28:58,Start
2,Case1,Create Purchase Requisition,2011-01-01 02:28:58,2011-01-01 02:44:02,Create Purchase Requisition
3,Case1,Create Request for Quotation,2011-01-01 02:57:37,2011-01-01 03:01:23,Create Request for Quotation
4,Case1,Analyze Request for Quotation,2011-01-12 20:34:39,2011-01-12 20:41:21,Analyze Request for Quotation
5,Case1,Send Request for Quotation to Supplier,2011-01-13 06:18:22,2011-01-13 06:47:02,Send Request for Quotation to Supplier
...,...,...,...,...,...
2208,Case122,Create Request for Quotation,2011-01-06 13:06:42,2011-01-06 13:08:07,Create Request for Quotation
2209,Case122,Analyze Request for Quotation,2011-01-09 12:07:56,2011-01-09 12:35:12,Analyze Request for Quotation
2210,Case122,Amend Request for Quotation,2011-01-09 13:35:36,2011-01-09 13:43:03,Amend Request for Quotation


In [113]:
g.to_csv(f'gen_diffu_time_{file_name}_4.csv', index=False)